In [23]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import os
from dotenv import load_dotenv
import json

In [2]:
load_dotenv()
# Get the API key from the environment variable
API_TOKEN = os.getenv('Hugging_Token')

In [50]:
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
def query(data):
  
  processed_news = []


  for item in data['body']:
    payload = item['title']   
    response = requests.post(API_URL, headers=headers, json=payload)
    response = response.json()


    for i in range(len(data["body"])):
      sentiments = response[i]

      # Adding sentiment scores to the current item
      data["body"][i]["negative"] = sentiments[0]["score"]
      data["body"][i]["neutral"] = sentiments[1]["score"]
      data["body"][i]["positive"] = sentiments[2]["score"]

  return data

In [51]:
test = {
  "statusCode": 200,
  "body": [
    {
      "date": "2024-05-30",
      "title": "European bitcoin ETPs suffer mounting outflows"
    }
  ]
}

data = query(test)
print(data)

{'statusCode': 200, 'body': [{'date': '2024-05-30', 'title': 'European bitcoin ETPs suffer mounting outflows', 'negative': 0.9985144734382629, 'neutral': 0.0009222572552971542, 'positive': 0.0005632292595691979}]}


In [ ]:
load_dotenv()
# Get the API key from the environment variable
API_TOKEN = os.getenv('Hugging_Token')
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
def query(data):
  
  processed_news = []


  for item in data['body']:
    payload = item['title']   
    response = requests.post(API_URL, headers=headers, json=payload)
    response = response.json()


    for i in range(len(data["body"])):
      sentiments = response[i]

      # Adding sentiment scores to the current item
      data["body"][i]["negative"] = sentiments[0]["score"]
      data["body"][i]["neutral"] = sentiments[1]["score"]
      data["body"][i]["positive"] = sentiments[2]["score"]

  return data

test = {
  "statusCode": 200,
  "body": [
    {
      "date": "2024-05-30",
      "title": "European bitcoin ETPs suffer mounting outflows"
    }
  ]
}

data = query(test)
print(data)

In [2]:
import json
import os
import requests

# Fetch the API token from environment variables
API_TOKEN = os.getenv('Hugging_Token')
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

def query(payload):
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def analyze_sentiment(news_title):
    response = query({"inputs": news_title})
    print(f"API response for title '{news_title}': {json.dumps(response, indent=2)}")  # Detailed debug output
    if response and isinstance(response, list) and len(response) > 0:
        # Check the structure of the response before attempting to access it
        if all(isinstance(item, dict) for item in response):
            sentiments = {item['label']: item['score'] for item in response}
            return sentiments
        else:
            print(f"Unexpected response format for title '{news_title}': {response}")
    return None

def lambda_handler(event, context):
    news_data = event.get('body', [])
    processed_news = []

    for news in news_data:
        title = news.get("title", "")
        date = news.get("date", "")
        if title and date:
            sentiment = analyze_sentiment(title)
            print(f"Sentiment for title '{title}': {sentiment}")  # Debugging output
            if sentiment:
                processed_news.append({
                    "date": date,
                    "title": title,
                    "negative": sentiment.get('negative', 0),
                    "positive": sentiment.get('positive', 0),
                    "neutral": sentiment.get('neutral', 0)
                })
    
    return {
        'statusCode': 200,
        'body': json.dumps(processed_news)
    }

# Example test event
event = {
    "body": [
        {
            "date": "2024-05-30",
            "title": "European bitcoin ETPs suffer mounting outflows"
        }
    ]
}

# Simulating lambda handler execution for testing
context = {}  # Simulated context object
response = lambda_handler(event, context)
print(response)


API response for title 'European bitcoin ETPs suffer mounting outflows': [
  [
    {
      "label": "negative",
      "score": 0.9985144734382629
    },
    {
      "label": "neutral",
      "score": 0.0009222572552971542
    },
    {
      "label": "positive",
      "score": 0.0005632292595691979
    }
  ]
]
Unexpected response format for title 'European bitcoin ETPs suffer mounting outflows': [[{'label': 'negative', 'score': 0.9985144734382629}, {'label': 'neutral', 'score': 0.0009222572552971542}, {'label': 'positive', 'score': 0.0005632292595691979}]]
Sentiment for title 'European bitcoin ETPs suffer mounting outflows': None
{'statusCode': 200, 'body': '[]'}


In [4]:
##Original

In [ ]:
import json
import os
import requests



API_TOKEN = os.getenv('Hugging_Token')
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
    


def analyze_sentiment(news_title):
    response = query(news_title)
    if response and isinstance(response, list) and len(response) > 0:
        sentiments = {item['label']: item['score'] for item in response}
        return sentiments
    return None

def lambda_handler(event, context):
    news_data = event['body']
    processed_news = []

    for news in news_data:
        sentiment = analyze_sentiment(news["title"])  # Use get() method to handle missing keys
        if sentiment:
            processed_news.append({
            "date": news["date"],
            "title": news["title"],
            "negative": sentiment.get('negative', 0),
            "positive": sentiment.get('positive', 0),
            "neutral": sentiment.get('neutral', 0)
        })
    
    return {
        'statusCode': 200,
        'body': json.dumps(processed_news)
    }


# For local testing
#if __name__ == "__main__":
    #event = {
        #"body": json.dumps([
            #{"date": "2024-06-10", "title": "European bitcoin ETPs suffer mounting outflows"},
            #{"date": "2024-06-11", "title": "Bitcoin price surges after regulatory approval"},
            #{"date": "2024-06-12", "title": "Cryptocurrency market experiences significant downturn"}
        #])
    #}
    #context = {}
    #result = lambda_handler(event, context)
    #print(result)


In [7]:
import json
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the API token from the environment variable
API_TOKEN = os.getenv('Hugging_Token')
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

def lambda_handler(event, context):
    processed_news = []

    for item in event['body']:
        payload = {"inputs": item['title']}
        response = requests.post(API_URL, headers=headers, json=payload)
        
        try:
            response = response.json()
            print(f"API response for title '{item['title']}': {json.dumps(response, indent=2)}")  # Debug output
            if response and isinstance(response, list) and len(response) > 0:
                if isinstance(response[0], dict):
                    sentiments = {entry['label']: entry['score'] for entry in response}

                    item['negative'] = sentiments.get('negative', 0)
                    item['neutral'] = sentiments.get('neutral', 0)
                    item['positive'] = sentiments.get('positive', 0)
                else:
                    print(f"Unexpected format in response for title '{item['title']}': {response}")
            else:
                print(f"Unexpected response format: {response}")
        except json.JSONDecodeError:
            print(f"Failed to decode JSON response: {response}")
        except KeyError as e:
            print(f"Key error: {e} in response: {response}")
        except TypeError as e:
            print(f"Type error: {e} in response: {response}")

        processed_news.append(item)

    return {
        'statusCode': 200,
        'body': json.dumps(processed_news)
    }

# Example test event
event = {
    "body": [
        {
            "date": "2024-05-30",
            "title": "European bitcoin ETPs suffer mounting outflows"
        }
    ]
}

# Simulating lambda handler execution for testing
context = {}  # Simulated context object
response = lambda_handler(event, context)
print(response)


API response for title 'European bitcoin ETPs suffer mounting outflows': [
  [
    {
      "label": "negative",
      "score": 0.9985144734382629
    },
    {
      "label": "neutral",
      "score": 0.0009222572552971542
    },
    {
      "label": "positive",
      "score": 0.0005632292595691979
    }
  ]
]
Unexpected format in response for title 'European bitcoin ETPs suffer mounting outflows': [[{'label': 'negative', 'score': 0.9985144734382629}, {'label': 'neutral', 'score': 0.0009222572552971542}, {'label': 'positive', 'score': 0.0005632292595691979}]]
{'statusCode': 200, 'body': '[{"date": "2024-05-30", "title": "European bitcoin ETPs suffer mounting outflows"}]'}


In [9]:
import json
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the API token from the environment variable
API_TOKEN = os.getenv('Hugging_Token')
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

def lambda_handler(event, context):
    processed_news = []

    for item in event['body']:
        payload = {"inputs": item['title']}
        response = requests.post(API_URL, headers=headers, json=payload)
        
        try:
            response = response.json()
            print(f"API response for title '{item['title']}': {json.dumps(response, indent=2)}")  # Debug output
            if response and isinstance(response, list) and len(response) > 0:
                if isinstance(response[0], dict) and 'label' in response[0] and 'score' in response[0]:
                    sentiments = {entry['label']: entry['score'] for entry in response}

                    item['negative'] = sentiments.get('negative', 0)
                    item['neutral'] = sentiments.get('neutral', 0)
                    item['positive'] = sentiments.get('positive', 0)
                else:
                    print(f"Unexpected format in response for title '{item['title']}': {response}")
            else:
                print(f"Unexpected response format: {response}")
        except json.JSONDecodeError:
            print(f"Failed to decode JSON response: {response}")
        except KeyError as e:
            print(f"Key error: {e} in response: {response}")
        except TypeError as e:
            print(f"Type error: {e} in response: {response}")

        processed_news.append(item)

    return {
        'statusCode': 200,
        'body': json.dumps(processed_news)
    }

# Example test event
event = {
    "body": [
        {
            "date": "2024-06-10",
            "title": "European bitcoin ETPs suffer mounting outflows"
        }
    ]
}

# Simulating lambda handler execution for testing
context = {}  # Simulated context object
response = lambda_handler(event, context)
print(response)


API response for title 'European bitcoin ETPs suffer mounting outflows': [
  [
    {
      "label": "negative",
      "score": 0.9985144734382629
    },
    {
      "label": "neutral",
      "score": 0.0009222572552971542
    },
    {
      "label": "positive",
      "score": 0.0005632292595691979
    }
  ]
]
Unexpected format in response for title 'European bitcoin ETPs suffer mounting outflows': [[{'label': 'negative', 'score': 0.9985144734382629}, {'label': 'neutral', 'score': 0.0009222572552971542}, {'label': 'positive', 'score': 0.0005632292595691979}]]
{'statusCode': 200, 'body': '[{"date": "2024-06-10", "title": "European bitcoin ETPs suffer mounting outflows"}]'}


In [10]:
import json
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the API token from the environment variable
API_TOKEN = os.getenv('Hugging_Token')
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

def lambda_handler(event, context):
    processed_news = []

    for item in event['body']:
        payload = {"inputs": item['title']}
        try:
            response = requests.post(API_URL, headers=headers, json=payload)
            response.raise_for_status()
            response = response.json()
            
            if response and isinstance(response, list) and len(response) > 0:
                sentiments = {entry['label']: entry['score'] for entry in response}
                item['negative'] = sentiments.get('negative', 0)
                item['neutral'] = sentiments.get('neutral', 0)
                item['positive'] = sentiments.get('positive', 0)
            else:
                print(f"Unexpected response format: {response}")
        except requests.exceptions.RequestException as e:
            print(f"HTTP request failed: {e}")
        except json.JSONDecodeError:
            print(f"Failed to decode JSON response: {response}")
        except KeyError as e:
            print(f"Key error: {e} in response: {response}")
        except TypeError as e:
            print(f"Type error: {e} in response: {response}")

        processed_news.append(item)

    return {
        'statusCode': 200,
        'body': json.dumps(processed_news)
    }

# Example test event
event = {
    "body": [
        {
            "date": "2024-06-10",
            "title": "European bitcoin ETPs suffer mounting outflows"
        }
    ]
}

# Simulating lambda handler execution for testing
context = {}  # Simulated context object
response = lambda_handler(event, context)
print(response)


Type error: list indices must be integers or slices, not str in response: [[{'label': 'negative', 'score': 0.9985144734382629}, {'label': 'neutral', 'score': 0.0009222572552971542}, {'label': 'positive', 'score': 0.0005632292595691979}]]
{'statusCode': 200, 'body': '[{"date": "2024-06-10", "title": "European bitcoin ETPs suffer mounting outflows"}]'}


In [12]:
import json
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the API token from the environment variable
API_TOKEN = os.getenv('Hugging_Token')
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

def lambda_handler(event, context):
    processed_news = []

    for item in event['body']:
        payload = {"inputs": item['title']}
        try:
            response = requests.post(API_URL, headers=headers, json=payload)
            response.raise_for_status()
            response = response.json()

            # Check if the response is a list of lists
            if isinstance(response, list) and len(response) == 1 and isinstance(response[0], list):
                response = response[0]  # Extract the inner list
                
            if response and isinstance(response, list) and len(response) > 0:
                sentiments = {entry['label']: entry['score'] for entry in response}
                item['negative'] = sentiments.get('negative', 0)
                item['neutral'] = sentiments.get('neutral', 0)
                item['positive'] = sentiments.get('positive', 0)
            else:
                print(f"Unexpected response format: {response}")
        except requests.exceptions.RequestException as e:
            print(f"HTTP request failed: {e}")
        except json.JSONDecodeError:
            print(f"Failed to decode JSON response: {response}")
        except KeyError as e:
            print(f"Key error: {e} in response: {response}")
        except TypeError as e:
            print(f"Type error: {e} in response: {response}")

        processed_news.append(item)

    return {
        'statusCode': 200,
        'body': json.dumps(processed_news)
    }

# Example test event
event = {
    "body": [
        {
            "date": "2024-06-10",
            "title": "European bitcoin ETPs suffer mounting outflows"
        },
    {
      "date": "2024-05-24",
      "title": "British-Chinese bitcoin money launderer jailed for over 6 years"
    }
    ]
}



# Simulating lambda handler execution for testing
context = {}  # Simulated context object
response = lambda_handler(event, context)
print(response)


{'statusCode': 200, 'body': '[{"date": "2024-06-10", "title": "European bitcoin ETPs suffer mounting outflows", "negative": 0.9985144734382629, "neutral": 0.0009222572552971542, "positive": 0.0005632292595691979}, {"date": "2024-05-24", "title": "British-Chinese bitcoin money launderer jailed for over 6 years", "negative": 0.04242394119501114, "neutral": 0.9568539261817932, "positive": 0.0007221141131594777}]'}


In [ ]:
for item in event['body']:
    payload = {"inputs": item['title']}
    print("Payload:", payload)  # Print the payload before making the request
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        # Rest of your code...


In [13]:
import json
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the API token from the environment variable
API_TOKEN = os.getenv('Hugging_Token')
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

def lambda_handler(event, context):
    processed_news = []

    for item in event['body']:
        payload = {"inputs": item['title']}
        print("Payload:", payload)  # Print the payload before making the request
        try:
            response = requests.post(API_URL, headers=headers, json=payload)
            response.raise_for_status()
            response = response.json()

            # Check if the response is a list of lists
            if isinstance(response, list) and len(response) == 1 and isinstance(response[0], list):
                response = response[0]  # Extract the inner list
                
            if response and isinstance(response, list) and len(response) > 0:
                sentiments = {entry['label']: entry['score'] for entry in response}
                item['negative'] = sentiments.get('negative', 0)
                item['neutral'] = sentiments.get('neutral', 0)
                item['positive'] = sentiments.get('positive', 0)
            else:
                print(f"Unexpected response format: {response}")
        except requests.exceptions.RequestException as e:
            print(f"HTTP request failed: {e}")
        except json.JSONDecodeError:
            print(f"Failed to decode JSON response: {response}")
        except KeyError as e:
            print(f"Key error: {e} in response: {response}")
        except TypeError as e:
            print(f"Type error: {e} in response: {response}")

        processed_news.append(item)

    return {
        'statusCode': 200,
        'body': json.dumps(processed_news)
    }

# Example test event
event = {
    "body": [
        {
            "date": "2024-06-10",
            "title": "European bitcoin ETPs suffer mounting outflows"
        },
    {
      "date": "2024-05-24",
      "title": "British-Chinese bitcoin money launderer jailed for over 6 years"
    }
    ]
}



# Simulating lambda handler execution for testing
context = {}  # Simulated context object
response = lambda_handler(event, context)
print(response)


Payload: {'inputs': 'European bitcoin ETPs suffer mounting outflows'}
Payload: {'inputs': 'British-Chinese bitcoin money launderer jailed for over 6 years'}
{'statusCode': 200, 'body': '[{"date": "2024-06-10", "title": "European bitcoin ETPs suffer mounting outflows", "negative": 0.9985144734382629, "neutral": 0.0009222572552971542, "positive": 0.0005632292595691979}, {"date": "2024-05-24", "title": "British-Chinese bitcoin money launderer jailed for over 6 years", "negative": 0.04242394119501114, "neutral": 0.9568539261817932, "positive": 0.0007221141131594777}]'}


In [ ]:

def lambda_handler(event, context):
    processed_news = []

    # Debugging: Print event body to inspect its format
    print("Event Body:", event['body'])

    for item in event['body']:
        payload = {"inputs": item['title']}
        print("Payload:", payload)  # Print the payload before making the request
        try:
            response = requests.post(API_URL, headers=headers, json=payload)
            response.raise_for_status()
            response = response.json()

            # Debugging: Print response from Hugging Face API
            print("API Response:", response)

            # Check if the response is a list of lists
            if isinstance(response, list) and len(response) == 1 and isinstance(response[0], list):
                response = response[0]  # Extract the inner list
                
            if response and isinstance(response, list) and len(response) > 0:
                sentiments = {entry['label']: entry['score'] for entry in response}
                item['negative'] = sentiments.get('negative', 0)
                item['neutral'] = sentiments.get('neutral', 0)
                item['positive'] = sentiments.get('positive', 0)
            else:
                print(f"Unexpected response format: {response}")
        except requests.exceptions.RequestException as e:
            print(f"HTTP request failed: {e}")
        except json.JSONDecodeError:
            print(f"Failed to decode JSON response: {response}")
        except KeyError as e:
            print(f"Key error: {e} in response: {response}")
        except TypeError as e:
            print(f"Type error: {e} in response: {response}")

        processed_news.append(item)

    return {
        'statusCode': 200,
        'body': json.dumps(processed_news)
    }


In [ ]:
## Original
import json
import os
import psycopg2

# Database connection parameters
DATABASE_URL = os.getenv('DataBase_Url')

def save_to_database(news_data):
    try:
        # Connect to the database
        conn = psycopg2.connect(DATABASE_URL)
        
        # Insert the Bitcoin news into the database
        cursor = conn.cursor()
        for item in news_data:
            cursor.execute('''
                INSERT INTO sentiment_bitcoin_news (date, title, negative, positive, neutral)
                VALUES (%s, %s, %s, %s, %s)
                ON CONFLICT(title) DO NOTHING
            ''', (item['date'], item['title'], item['negative'], item['positive'], item['neutral']))
        
        conn.commit()
        cursor.close()
        conn.close()
        
        return {
            'statusCode': 200,
            'body': json.dumps('Data inserted successfully')
        }
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        return {
            'statusCode': 500,
            'body': json.dumps(f'Error inserting Bitcoin data: {e}')
        }

def lambda_handler(event, context):
    print("Event body:", event['body'])  # Print the event body
    #news_data = json.loads(event['body'])
    #news_data = json.loads(event['body'])
    return save_to_database(event['body'])

# For local testing
#if __name__ == "__main__":
    #event = {
        #"body": json.dumps([
            #{
                #"date": "2024-06-10",
                #"title": "European bitcoin ETPs suffer mounting outflows",
                #"negative": 0.85,
                #"positive": 0.1,
                #"neutral": 0.05
            #},
            #{
               ## "date": "2024-06-11",
                #"title": "Bitcoin price surges after regulatory approval",
                #"negative": 0.1,
                #"positive": 0.85,
                #"neutral": 0.05
            #},
            #{
                #"date": "2024-06-12",
                #"title": "Cryptocurrency market experiences significant downturn",
                #"negative": 0.78,
                #"positive": 0.15,
                #"neutral": 0.07
            #}
        #])
    #}
    #context = {}
    #result = lambda_handler(event, context)
    #print(result)


In [ ]:
import json
import os
import requests

# Get the API token from the environment variable
API_TOKEN = os.getenv('Hugging_Token')
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

def lambda_handler(event, context):
    # Extract the event from the responsePayload
    response_payload = event.get('responsePayload')
    if response_payload:
        processed_news = []  # Correct indentation here

        # Debugging: Print event body to inspect its format
        print("Event Body:", response_payload['body'])

        for item in response_payload['body']:
            payload = {"inputs": item['title']}
            print("Payload:", payload)  # Print the payload before making the request
            response = requests.post(API_URL, headers=headers, json=payload)
            response_json = response.json()

            # Debugging: Print response from Hugging Face API
            print("API Response:", response_json)

            # Check if the response is a list of lists
            if isinstance(response_json, list) and len(response_json) == 1 and isinstance(response_json[0], list):
                response_json = response_json[0]  # Extract the inner list
                    
            if isinstance(response_json, list) and len(response_json) > 0:
                sentiments = {entry['label']: entry['score'] for entry in response_json}
                item['negative'] = sentiments.get('negative', 0)
                item['positive'] = sentiments.get('positive', 0)
                item['neutral'] = sentiments.get('neutral', 0)
            else:
                print(f"Unexpected response format: {response_json}")

            processed_news.append(item)

        return {
            'statusCode': 200,
            'body': json.dumps(processed_news)
        }
    else:
        return {
            'statusCode': 400,
            'body': json.dumps('No responsePayload found in the event')
        }
